In [ ]:
### Put ubc_data in drive as ubc_data_relevant
### Spoiler: full dataset is too big for ram (TODO: better dataloader that doesnt need to load into ram)
### Data filter for only users from relevant_users can be achived with something like (untested):

# import pandas as pd

# # Load the dataframes from parquet files
# add_to_cart = pd.read_parquet('../ubc_data/add_to_cart.parquet')
# page_visit = pd.read_parquet('../ubc_data/page_visit.parquet')
# product_buy = pd.read_parquet('../ubc_data/product_buy.parquet')
# product_properties = pd.read_parquet('../ubc_data/product_properties.parquet')
# remove_from_cart = pd.read_parquet('../ubc_data/remove_from_cart.parquet')
# search_query = pd.read_parquet('../ubc_data/search_query.parquet')

# full_recsys = RecSysData(
#     add_to_cart,
#     page_visit,
#     product_buy,
#     product_properties,
#     remove_from_cart,
#     search_query
#   )

# import numpy as np

# # Load the relevant users from the numpy file
# relevant_users = np.load('./ubc_data/input/relevant_clients.npy')
# relevant_users_set = set(relevant_users)

# relevant_users_data = full_recsys.sampled_subset(relevant_users_set, reset_index=False)
# relevant_users_data.print_datasets_stats_nicely("Relevant Users Dataset")

# relevant_users_data.save_parquet('./ubc_data_relevant')

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
! ls /content/drive/MyDrive/ubc_data_relevant/
ubc_data_path = '/content/drive/MyDrive/ubc_data_relevant'

add_to_cart.parquet  product_buy.parquet	 search_query.parquet
input		     product_properties.parquet  target
page_visit.parquet   remove_from_cart.parquet


In [131]:
# ! rm -rf recsys2025_challenge

In [132]:
! test -f recsys2025_challenge || git clone --recurse-submodules --remote-submodules https://github.com/zmrocze/recsys2025_challenge.git

Cloning into 'recsys2025_challenge'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 112 (delta 64), reused 88 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (112/112), 816.65 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Submodule 'recsys2025' (https://github.com/Synerise/recsys2025.git) registered for path 'recsys2025'
Cloning into '/content/recsys2025_challenge/recsys2025'...
remote: Enumerating objects: 71, done.        
remote: Counting objects: 100% (19/19), done.        
remote: Compressing objects: 100% (11/11), done.        
remote: Total 71 (delta 9), reused 8 (delta 8), pack-reused 52 (from 1)        
Receiving objects: 100% (71/71), 47.95 KiB | 380.00 KiB/s, done.
Resolving deltas: 100% (20/20), done.
Submodule path 'recsys2025': checked out '25ec48089a8e3516928b9a402dea4c8dba6753d2'


In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
import sys

# Below paths: bad but managable

module_path = os.path.abspath(os.path.join('./recsys2025_challenge'))
sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('./recsys2025_challenge/recsys2025'))
sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('./recsys2025_challenge/src/our_lib'))
# print(module_path)
sys.path.append(module_path)

# sys.path.insert(1, "./recsys2025_challenge/src/our_lib")

# from our_lib import *
# from src.our_lib import RecSysData
# from our_lib import split_data, create_baseline_embeddings, contest_training
# from our_lib import validate_and_load_embeddings

In [6]:
! pip install pytorch-lightning torchmetrics neptune pyarrow
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.9/487.9 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:

# load data

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
# load everything
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchmetrics as tm

# Load propensity data from .npy files
propensity_sku = np.load(f'{ubc_data_path}/target/propensity_sku.npy')
propensity_category = np.load(f'{ubc_data_path}/target/propensity_category.npy')
relevant_users = np.load(f'{ubc_data_path}/input/relevant_clients.npy')

print("Propensity SKU shape:", propensity_sku.shape)
print("Propensity Category shape:", propensity_category.shape)

Propensity SKU shape: (100,)
Propensity Category shape: (100,)


In [22]:
from src.our_lib import RecSysData

data = RecSysData.read_parquet(ubc_data_path)
data

RecSysData(
  add_to_cart=2654264,
  page_visit=41919050,
  product_buy=1218423,
  product_properties=831550,
  remove_from_cart=1138370,
  search_query=4212745,
  all users: 1000000,
  all products: 831550
  all urls: 6171078,
)

In [23]:
# drop duplicates
data.add_to_cart = data.add_to_cart.drop_duplicates(subset=['sku'])
data.product_buy = data.product_buy.drop_duplicates(subset=['sku'])
data.remove_from_cart = data.remove_from_cart.drop_duplicates(subset=['sku'])

In [24]:
# prompt: leave only those users which added to cart at least 10 items, and only items which were added by at least 10 users

user_counts = data.add_to_cart.groupby('client_id').size()
user_counts_remove = data.remove_from_cart.groupby('client_id').size()
user_counts_buy = data.product_buy.groupby('client_id').size()

# prompt: we calculated how many rows with given user there are in add_to_cart, remove_from_cart and product_buy (user_counts, user_counts_remove and user_counts_buy). Now join them into a single table, and then calculate how many interactions total does every user have and take only users with at least 10 interactions

# Combine the counts into a single DataFrame
user_interactions = pd.concat([user_counts, user_counts_remove, user_counts_buy], axis=1).fillna(0)
user_interactions.columns = ['add_to_cart', 'remove_from_cart', 'product_buy']

# Calculate the total interactions for each user
user_interactions['total_interactions'] = user_interactions.sum(axis=1)

In [25]:
# Filter for users with at least 10 total interactions
users_with_enough_interactions = user_interactions[user_interactions['total_interactions'] >= 10].index
print(f"Users with enough interactions left: {len(users_with_enough_interactions)}")
data.add_to_cart = data.add_to_cart[data.add_to_cart['client_id'].isin(users_with_enough_interactions)].reset_index(drop=True)
data.remove_from_cart = data.remove_from_cart[data.remove_from_cart['client_id'].isin(users_with_enough_interactions)].reset_index(drop=True)
data.product_buy = data.product_buy[data.product_buy['client_id'].isin(users_with_enough_interactions)].reset_index(drop=True)

unique_add_to_cart = unique_edges(data.add_to_cart)
unique_remove_from_cart = unique_edges(data.remove_from_cart)
unique_product_buy = unique_edges(data.product_buy)

unique_add_to_cart, unique_remove_from_cart, unique_product_buy

Users with enough interactions left: 26656


(        client_id            timestamp      sku
 0        19160733  2022-08-23 18:05:55   858011
 1        19160733  2022-08-23 18:10:10   569264
 2        19160733  2022-08-23 18:10:45  1035744
 3        19160733  2022-08-23 18:11:05   363476
 4        19160733  2022-08-23 18:15:45  1014020
 ...           ...                  ...      ...
 302465   23585940  2022-08-12 08:51:55  1365365
 302466   23585940  2022-08-12 08:52:20   841711
 302467   23585940  2022-08-12 09:01:45   167812
 302468   23585940  2022-08-12 09:05:10  1037763
 302469   23585940  2022-08-12 09:05:35   383521
 
 [302470 rows x 3 columns],
         client_id            timestamp      sku
 0        19160733  2022-08-23 18:59:05  1217064
 1        19160733  2022-08-23 19:00:25   446526
 2        23001812  2022-07-06 10:24:59   338985
 3        23001812  2022-07-06 10:41:30   939511
 4        23001812  2022-07-06 10:41:30  1103060
 ...           ...                  ...      ...
 219851   15968646  2022-09-26 17:34:00

In [26]:
# prompt: get unique users from all 3 tables

unique_users_add_to_cart = unique_add_to_cart['client_id'].unique()
unique_users_remove_from_cart = unique_remove_from_cart['client_id'].unique()
unique_users_product_buy = unique_product_buy['client_id'].unique()

users = pd.concat([
    pd.Series(unique_users_add_to_cart),
    pd.Series(unique_users_remove_from_cart),
    pd.Series(unique_users_product_buy)
]).unique()

unique_items_add_to_cart = unique_add_to_cart['sku'].unique()
unique_items_remove_from_cart = unique_remove_from_cart['sku'].unique()
unique_items_product_buy = unique_product_buy['sku'].unique()

items = pd.concat([
    pd.Series(unique_items_add_to_cart),
    pd.Series(unique_items_remove_from_cart),
    pd.Series(unique_items_product_buy)
]).unique()

print(f"Total unique users across all tables: {len(users)}")
print(f"Total unique items across all tables: {len(items)}")
# print(all_unique_users)

Total unique users across all tables: 26656
Total unique items across all tables: 365893


# training gat

In [133]:
# imports
# from src.our_lib import NodeIdMap, JustGAT, RecGAT, DotproductEdgePredictor, LinearEdgePredictor, BprLossLoader
# from src.our_lib import test_out, l2_reg, train_test_split_pos_edges, int_tensor, unique_edges
from src.our_lib import NodeIdMap, JustGAT, RecGAT, DotproductEdgePredictor, LinearEdgePredictor, BprLossLoader, test_out, l2_reg, train_test_split_pos_edges, int_tensor, unique_edges, loss_f, create_target_from_edge_index, BprTraining, device


In [134]:
### NOTE!:
### There are 2 types of ids: product has sku    and the id of a product node in graph (can exchange with node_id_map.item_of_id/id_of_item).
#                             user has client_id and the id of a user    node in graph

node_id_map = NodeIdMap(users, items)
all_edge_index = node_id_map.make_edges(unique_add_to_cart.client_id.values, unique_add_to_cart.sku.values).to(device=device)
all_edge_index_remove = node_id_map.make_edges(unique_remove_from_cart.client_id.values, unique_remove_from_cart.sku.values).to(device=device)
all_edge_index_buy = node_id_map.make_edges(unique_product_buy.client_id.values, unique_product_buy.sku.values).to(device=device)

N = node_id_map.N
print(f"Total nodes in user-item graph: {N}")
print("all_edge_index.shape", all_edge_index.shape)
print("all_edge_index_remove.shape", all_edge_index_remove.shape)
print("all_edge_index_buy.shape", all_edge_index_buy.shape)

train_edges, test_edges = train_test_split_pos_edges(all_edge_index, test_size=0.3, random_state=42)

# Create one-hot encoded vectors for each edge type
train_edge_attr_add_to_cart = torch.tensor([[1, 0, 0]], dtype=torch.float).repeat(train_edges.size(1), 1).to(device)
edge_attr_remove_from_cart = torch.tensor([[0, 1, 0]], dtype=torch.float).repeat(all_edge_index_remove.size(1), 1).to(device)
edge_attr_product_buy = torch.tensor([[0, 0, 1]], dtype=torch.float).repeat(all_edge_index_buy.size(1), 1).to(device)

print("train_edges.shape", train_edges.shape)

Total nodes in user-item graph: 392549
all_edge_index.shape torch.Size([2, 302470])
all_edge_index_remove.shape torch.Size([2, 219856])
all_edge_index_buy.shape torch.Size([2, 118509])
train_edges.shape torch.Size([2, 211729])


In [135]:
neg_samples = 15
batch_size = 2048
### these are edges used for loss calculation
### and they define the graph (maybe together with product_buy, remove_from_cart)
train_loader = BprLossLoader(
    edge_index=train_edges,
    trg_index_range=(node_id_map.n_users, node_id_map.N), # node index range for all items (used to sample negative items)
    batch_size=batch_size,
    neg_samples=neg_samples,
    random_state=42,
    device=device
)

test_loader = BprLossLoader(
  edge_index=test_edges,
  trg_index_range=(node_id_map.n_users, node_id_map.N),  # node index range for all items (used to sample negative items)
  batch_size=batch_size,
  neg_samples=neg_samples,
  random_state=42,
  device=device
)


In [136]:
### Making a set of 300 items to test auroc on. (Why not use propensity_items? Because many of them may not be in our data. Sth todo)

# Get 300 random items that added to cart at least one item in test set (so that target is not all 0s)

n_sample_items = min(300, len(items))
items_with_edge = np.array(list(set( (node_id_map.item_of_id[i.item()] for i in set(test_edges[1, :]) ))), dtype=np.long)
sampled_items = np.random.choice(items_with_edge, n_sample_items, replace=False)
# create target for AUROC calculation
full_test_target = create_target_from_edge_index(
    node_id_map,
    node_id_map.n_users,
    sampled_items,
    test_edges
).to(device=device)
print("Full test target shape:", full_test_target.shape)
print("Full test target 1s:", full_test_target.sum())

Full test target shape: torch.Size([26656, 300])
Full test target 1s: tensor(300, device='cuda:0')


In [137]:
def remake_model_if_loss_nan(new_model):
  model = new_model()
  # model.reinit_weights(type='uniform', a=0.001)
  # Take 4096 random user and item indices for testing
  n_test = min(4096, model.recgat.node_id_map.n_users, model.recgat.node_id_map.n_items)
  user_indices = np.random.choice(model.recgat.node_id_map.n_users, size=n_test)
  item_indices = np.random.choice(model.recgat.node_id_map.n_items, size=n_test)
  test_user_emb = model.recgat.node_embeddings.weight[user_indices]
  test_item_emb = model.recgat.node_embeddings.weight[model.recgat.node_id_map.n_users + item_indices]

  loss = loss_f(model.recgat, model.edge_predictor, test_user_emb, test_item_emb)
  while torch.isnan(loss) or torch.isneginf(loss) or torch.isinf(loss) or (loss.abs().item() > 10000.0):
    print(f"loss nan, trying out parameter reinitialization: {loss}")
    # model.reinit_weights(type='uniform', a=0.000001)
    model = new_model()
    loss = loss_f(model.recgat, model.edge_predictor, test_user_emb, test_item_emb)
  return model

In [138]:

def make_model():
  embedding_dim = 64
  edge_dim = 3
  ### define graph
  # new_recgat = lambda: RecGAT(users, items, embedding_dim=embedding_dim, edge_dim=edge_dim, num_layers=2, device=device, dropout=0.3, type='uniform', a=0.0000001)
  new_recgat = lambda: RecGAT(users, items, embedding_dim=embedding_dim, edge_dim=edge_dim, device=device, dropout=0.2, type='normal', a=0.001)
  recgat = new_recgat()
  recgat.add_edge_index(train_edges, edge_attr=train_edge_attr_add_to_cart)
  recgat.add_edge_index(all_edge_index_remove, edge_attr=edge_attr_remove_from_cart)
  recgat.add_edge_index(all_edge_index_buy, edge_attr=edge_attr_product_buy)

  ### edge predictor (here: only for add_to_cart edges. todo: predict more stuff (like next purchased category in next 14days from propensity categories) with more varied predictors and combine losses)
  edge_predictor = LinearEdgePredictor(embedding_dim=embedding_dim, dropout=0.2).to(device=device)

  model = BprTraining(
      recgat=recgat,
      edge_predictor=edge_predictor,
      lr=0.001,
      propensity_sku=sampled_items, # for auroc calc
      full_test_target=full_test_target, # for auroc calc
      # forward_gat_every_n = 20, ### tradeoff : speed vs training stability
      # this has to be fixed and work with n>1, this is slow, use big batch to make up for it:
      forward_gat_every_n = 1,
    )

  ### sillynes
  # model = remake_model_if_loss_nan(model)
  return model


In [ ]:

# sillyunes with nans
# model = remake_model_if_loss_nan(make_model)
model = make_model()

wandb_logger = WandbLogger(project="recsys2025", name="recgat_bpr_5per_relevant")

trainer = pl.Trainer(max_epochs=10, check_val_every_n_epoch=1
                     , logger=wandb_logger
                     )

# print("test_loader", test_loader)
# print("test_edges.shape", test_edges.shape)

wandb_logger.watch(model, log="all")
trainer.fit(model, train_loader, val_dataloaders=test_loader)

INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory ./recsys2025/930p0rdl/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOC

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]